In [159]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', 250)

In [160]:
!cd ..; python parse_tb.py

  0%|          | 0/216 [00:00<?, ?it/s]
2023-01-23 10:41:42.713916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-23 10:41:42.713953: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [161]:
res_df = pd.read_csv('../results/kglm.csv')

In [162]:
res_df = res_df[~res_df.exact_match.isna()]
res_df['test_split'] = res_df.test_path.apply(lambda x: 'ilpc-large' if 'large' in x else 'ilpc-small')
res_df = res_df[res_df.task_name == res_df.test_split]
res_df['iters'] = res_df.model_path.apply(lambda x: x.split('iters')[1].split('_')[0])

In [163]:
def get_scheduler(model_path, optim='adamw'):
    if pd.isna(model_path):
        return model_path 
    model_path = res_df.cpt_path[0]
    s = model_path.split('/lr')[1]
    scheduler = s[s.index('_') + 1:].split('_' + optim)[0]
    return scheduler

In [164]:
res_df['original_lr_scheduler'] = res_df.cpt_path.apply(get_scheduler)
res_df.loc[~res_df.cpt_path.isna(), 'lr_scheduler'] = res_df.original_lr_scheduler.dropna()

In [165]:
res_df['description'] = res_df.drop_description.apply(lambda x: not x)
res_df['neighborhood'] = res_df.drop_neighborhood.apply(lambda x: not x)

In [166]:
# show_cols = ['exact_match_entity', 'Hits@1', "Hits@1_pipeline", 'model_path']
# print(show_cols)
# for _, row in res_df.sort_values(gb_cols).iterrows():
#     print([row[col] for col in show_cols])

In [167]:
res_df.columns

Index(['task_name', 'from_pretrained', 'model_cfg', 'model_cls', 'model_type',
       'lr', 'batch_size', 'HVD_SIZE', 'lr_scheduler', 'input_seq_len',
       'model_path', 'drop_neighborhood', 'drop_description', 'cpt_path',
       'task_name.1', 'train_path', 'valid_path', 'test_path', 'index_path',
       'inference_entities_path', 'exact_match', 'exact_match_entity',
       'Hits@1', 'Hits@3', 'Hits@5', 'Hits@10', 'Hits@1_pipeline',
       'Hits@3_pipeline', 'Hits@5_pipeline', 'Hits@10_pipeline', 'test_split',
       'iters', 'original_lr_scheduler', 'description', 'neighborhood'],
      dtype='object')

In [168]:
# max
# gb_cols = ['task_name', 'from_pretrained', 'lr_scheduler', 'neighborhood', 'description']
gb_cols = ['task_name', 'from_pretrained', 'iters', 'description', 'neighborhood', 'lr_scheduler']
metric_cols = ['exact_match_entity', 'Hits@1', 'Hits@1_pipeline']

df = res_df[metric_cols + gb_cols]#.dropna()
group_df = df.groupby(gb_cols).max()[metric_cols]
# group_df['std'] = res_df.groupby(gb_cols).std()[['best_valid_f1']]
# group_df['n_runs'] = res_df.groupby(gb_cols).count()['exact_match_entity']
# group_df['runs'] = res_df.groupby(gb_cols).agg({'exact_match_entity':tuple})['exact_match_entity'].apply(lambda x: [round(i,2) for i in x])
group_df

exact_match_entity  \
task_name  from_pretrained iters  description neighborhood lr_scheduler                               
ilpc-large t5-base         150000 False       False        constant_with_warmup            8.027298   
                                              True         constant_with_warmup            8.901217   
                                  True        False        constant_with_warmup            7.314078   
                                              True         constant_with_warmup            8.900678   
                           20000  False       False        linear                          8.807934   
                                              True         linear                          8.218775   
                                  True        False        linear                          6.863708   
                                              True         linear                          8.096033   
                           50000  False       False        linear                          8.581393   
                                              True         linear                          9.008743   
           t5-small        150000 False       False        constant_with_warmup            7.585428   
                                                           linear                          7.584242   
                                              True         linear                          7.530209   
                           250000 False       False        constant_with_warmup            7.564594   
                                              True         constant_with_warmup            7.613715   
                                  True        False        constant_with_warmup            7.202475   
                                              True         constant_with_warmup            7.938924   
ilpc-small t5-base         100000 False       False        constant_with_warmup            5.099931   
                                              True         constant_with_warmup            5.513439   
                                  True        False        constant_with_warmup            0.000000   
                                              True         constant_with_warmup            5.099931   
                           20000  False       False        linear                          4.031702   
                                              True         linear                          5.254997   
                                  True        False        linear                          4.014473   
                                              True         linear                          5.186079   
                           50000  False       False        constant_with_warmup            5.530669   
                                                           linear                          5.547898   
                                              True         constant_with_warmup            5.375603   
                                  True        False        constant_with_warmup            5.272226   
                                                           linear                          5.444521   
                                              True         constant_with_warmup            5.099931   
           t5-small        100000 False       False        constant_with_warmup            4.927636   
                                              True         constant_with_warmup            4.359063   
                                  True        False        constant_with_warmup            4.600276   
                                              True         constant_with_warmup            5.082702   
                           30000  False       False        linear                          3.445899   
                                              True         linear                          4.255686   
                                  True        False        linea

In [134]:
# mean
gb_cols = ['task_name', 'from_pretrained', 'lr_scheduler', 'iters', 'neighborhood', 'description']
metric_cols = ['exact_match_entity', 'Hits@1', 'Hits@1_pipeline']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
# group_df['std'] = res_df.groupby(gb_cols).std()[['best_valid_f1']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['exact_match_entity']
group_df['runs'] = res_df.groupby(gb_cols).agg({'exact_match_entity':tuple})['exact_match_entity'].apply(lambda x: [round(i,2) for i in x])
group_df

exact_match_entity  \
task_name  from_pretrained lr_scheduler         iters  neighborhood description                       
ilpc-large t5-base         constant_with_warmup 150000 False        False                  7.349443   
                                                                    True                   7.093400   
                                                       True         False                  8.900948   
                                                                    True                   8.565551   
                           linear               50000  False        False                  8.581393   
                                                       True         False                  9.008743   
           t5-small        constant_with_warmup 150000 False        False                  7.585428   
                                                250000 False        False                  6.551069   
                                                                    True                   7.201558   
                                                       True         False                  6.282542   
                                                                    True                   7.934974   
                           linear               150000 False        False                  7.584242   
                                                       True         False                  7.530209   
ilpc-small t5-base         constant_with_warmup 100000 False        False                  5.099931   
                                                                    True                   0.000000   
                                                       True         False                  5.513439   
                                                                    True                   5.099931   
                                                50000  False        False                  5.530669   
                                                                    True                   5.272226   
                                                       True         False                  5.375603   
                                                                    True                   5.099931   
                           linear               50000  False        False                  5.547898   
                                                                    True                   5.444521   
           t5-small        constant_with_warmup 100000 False        False                  4.927636   
                                                                    True                   4.600276   
                                                       True         False                  4.359063   
                                                                    True                   5.082702   

                                                                                   Hits@1  \
task_name  from_pretrained lr_scheduler         iters  neighborhood description             
ilpc-large t5-base         constant_with_warmup 150000 False        False        0.082882   
                                                                    True         0.111010   
                                                       True         False        0.112480   
                                                                    True         0.092615   
                           linear               50000  False        False        0.109736   
                                                       True         False        0.115827   
           t5-small        constant_with_warmup 150000 False        False        0.113757   
                                                250000 False        False        0.112978   
                                                                    True         0.106050   
                                                       True         Fa

In [56]:
res_df.groupby(gb_cols

,task_name,from_pretrained,model_cfg,model_cls,model_type,lr,batch_size,HVD_SIZE,lr_scheduler,input_seq_len,...,exact_match_entity,Hits@1,Hits@3,Hits@5,Hits@10,Hits@1_pipeline,Hits@3_pipeline,Hits@5_pipeline,Hits@10_pipeline,test_split
0,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,5.099931,0.027912,0.033598,0.041178,0.053411,0.057547,0.061337,0.067195,0.077877,ilpc-small
1,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,12.716025,0.146995,0.166683,0.194226,0.219167,NaN,NaN,NaN,NaN,ilpc-small
2,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,14.458955,0.169040,0.191673,0.214405,0.242341,0.165897,0.184898,0.203800,0.224813,ilpc-small
3,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,14.458955,0.169040,0.191673,0.214405,0.242341,NaN,NaN,NaN,NaN,ilpc-small
4,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,5.513439,0.029290,0.036699,0.047209,0.058925,0.064611,0.069263,0.078050,0.087526,ilpc-small
6,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,64,1.0,constant_with_warmup,512,...,5.530669,0.072019,0.117850,0.132667,0.171089,NaN,NaN,NaN,NaN,ilpc-small
7,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,12.966418,0.138894,0.175815,0.193293,0.216516,NaN,NaN,NaN,NaN,ilpc-small
8,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,128,1.0,linear,512,...,5.099931,0.016713,0.031013,0.037216,0.046175,0.016713,0.031013,0.037216,0.046175,ilpc-small
9,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,16,1.0,constant_with_warmup,512,...,5.099931,0.073570,0.111819,0.136802,0.175224,NaN,NaN,NaN,NaN,ilpc-small
10,ilpc-small,t5-base,NaN,transformers:T5ForConditionalGeneration,encoder-decoder,0.000005,16,1.0,constant_with_warmup,512,...,5.375603,0.078567,0.137491,0.164197,0.208649,NaN,NaN,NaN,NaN,ilpc-small


In [ ]:
res_df.